In [3]:
#load necessary libraries
import pandas as pd
import numpy as np
#import pyspark.pandas as ps
import matplotlib.pyplot as plt

In [4]:
df_train = pd.read_csv('../input/human-activity-recognition-with-smartphones/test.csv')

In [5]:
df_test = pd.read_csv('../input/human-activity-recognition-with-smartphones/train.csv')